In [1]:
import os
# Install JDK 11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-11-openjdk-amd64"
# Install Spark 3.4.0
!curl -O https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!ln -s spark-3.4.0-bin-hadoop3 spark
!rm -f *.tgz
os.environ["SPARK_HOME"] = "/content/spark"
#Install findspark using pip to make pyspark importable as regular library
!pip -q install findspark
import findspark
findspark.init()

#importing pyspark
import pyspark
#importing sparksession
from pyspark.sql import SparkSession

#creating a sparksession object and providing appName
spark=SparkSession.builder.appName("local").getOrCreate()
sc = spark.sparkContext

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  370M  100  370M    0     0  5508k      0  0:01:08  0:01:08 --:--:-- 10.5M


## Creazione DataFrame tweets/users

In [3]:
tweets_df = spark.read.json('/content/drive/MyDrive/data/tweeter_json/tweets', multiLine=True)
users_df = spark.read.json('/content/drive/MyDrive/data/tweeter_json/users', multiLine=True)

In [4]:
print(tweets_df.count())
print(users_df.count())

10000
10000


In [5]:
tweets_df = tweets_df.dropDuplicates(['id']).cache()
users_df = users_df.dropDuplicates(['id']).cache()

In [6]:
print(tweets_df.count())
print(users_df.count())

8093
2790


In [7]:
tweets_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- mentions_id: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- mentions_screen_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



In [8]:
users_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favourites_counts: long (nullable = true)
 |-- follower_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)



## Creazione tabelle

In [9]:
tweets_df.createOrReplaceTempView('tweets')
users_df.createOrReplaceTempView('users')

1. Restituire lo `screen_name` dell'utente che ha inviato più tweets.



In [10]:
spark.sql('''
SELECT screen_name, COUNT(*) AS num_tweets
FROM tweets AS t JOIN users AS u ON t.user_id = u.id
GROUP BY screen_name
ORDER BY num_tweets DESC
LIMIT 1
''').show()

+--------------+----------+
|   screen_name|num_tweets|
+--------------+----------+
|PleasureEthics|       525|
+--------------+----------+



2. Creare un DataFrame `hashtags`, in cui ciascuna riga rappresenta un hashtag presente nel DataFrame `tweets`.

In [11]:
hashtags_df = spark.sql('''
SELECT lower(hashtag) AS hashtag
FROM (
    SELECT explode(hashtags) AS hashtag
    FROM tweets
)
''').cache()

hashtags_df.createOrReplaceTempView('hashtags')

hashtags_df.show()

+--------------------+
|             hashtag|
+--------------------+
|artificialintelli...|
|     machinelearning|
|        deeplearning|
|             bigdata|
|         datascience|
|      neuralnetworks|
|        deeplearning|
|                 iot|
|             bigdata|
|          government|
|                4org|
|     ombagencyreform|
|             govtech|
|                  hr|
|             bigdata|
|     makeyourownlane|
|         datascience|
|             dataviz|
|artificialintelli...|
|           marketing|
+--------------------+
only showing top 20 rows



3. Restituire i 100 hashtag più utilizzati, ordinati in base alla frequenza in modo decrescente.

In [12]:
spark.sql('''
SELECT hashtag, COUNT(*) AS frequenza
FROM hashtags
GROUP BY hashtag
ORDER BY frequenza DESC
LIMIT 100
''').show()

+--------------------+---------+
|             hashtag|frequenza|
+--------------------+---------+
|             bigdata|     8193|
|                  ai|     3204|
|                 iot|     1970|
|     machinelearning|     1500|
|         datascience|     1401|
|           analytics|     1329|
|                tech|      828|
|        deeplearning|      773|
|                  ml|      714|
|             fintech|      691|
|artificialintelli...|      676|
|         smartcities|      666|
|          blockchain|      633|
|                data|      630|
|            opendata|      560|
|digitaltransforma...|      504|
|               cloud|      503|
|       cybersecurity|      477|
|          technology|      460|
|          innovation|      427|
+--------------------+---------+
only showing top 20 rows



4. Restituire tutti i tweets inviati dai 10 utenti con il maggior numero di followers (ordinati in modo decrescente in base al numero di followers).

In [13]:
spark.sql('''
SELECT text, screen_name, follower_count
FROM (
    SELECT *
    FROM users
    ORDER BY follower_count DESC
    LIMIT 10
) AS top_users JOIN tweets AS t ON top_users.id = t.user_id
ORDER BY follower_count DESC
''').show(truncate=30)

+------------------------------+---------------+--------------+
|                          text|    screen_name|follower_count|
+------------------------------+---------------+--------------+
|#AI is not set and forget h...|     LouisSerge|       1061349|
|Facebook’s AI-created virtu...|     LouisSerge|       1061349|
|9 Instagram Story Ideas for...|     LouisSerge|       1061349|
|27 Social Media Marketing T...|     LouisSerge|       1061349|
|The Current State of Social...|     LouisSerge|       1061349|
|“Go Where Your Residents Ar...|     LouisSerge|       1061349|
|3 Simple Social Media Tips ...|     LouisSerge|       1061349|
|The 5 Commandments For a Hi...|     LouisSerge|       1061349|
|"Follow...create...and enga...|  2morrowknight|        937900|
|#Millennials are the larges...|  2morrowknight|        937900|
|What are the "5 Myths of Bi...|  2morrowknight|        937900|
|So, do you have the skills ...|  2morrowknight|        937900|
|INFOGRAPHIC: "The Most Acti...|  2morro


5. Trovare i 25 tweets con più retweets che sono stati inviati da utenti iscritti prima del 2014.

In [14]:
spark.sql('''
SELECT text, retweet_count
FROM tweets
WHERE user_id IN (
    SELECT id
    FROM users
    WHERE YEAR(created_at) < 2014
)
ORDER BY retweet_count DESC
LIMIT 25
''').show()

+--------------------+-------------+
|                text|retweet_count|
+--------------------+-------------+
|Which Machine Lea...|          474|
|The Moments Innov...|          222|
|Human-centered De...|          206|
|The Edge Computin...|          160|
|The Structured Pr...|          159|
|Happy 20th birthd...|          154|
|This machine will...|          147|
|LIKEtoBECOME a sp...|          131|
|[Infographic]\nAr...|          128|
|Data never sleeps...|          125|
|How will #Blockch...|           97|
|10 basic #tips fo...|           81|
|Introductory Guid...|           80|
|[interesting new ...|           79|
|An intuitive intr...|           67|
|#mustread // huge...|           57|
|Open Source #ETL ...|           55|
|8 #Industry40 fin...|           55|
|Top 5 #IoT applic...|           54|
|Injecting Ethical...|           52|
+--------------------+-------------+
only showing top 20 rows



6. Effettuare un word count sul testo dei tweets.

In [15]:
spark.sql('''
SELECT word, COUNT(*) AS word_count
FROM (
    SELECT explode(split(lower(text), " ")) AS word
    FROM tweets
)
GROUP BY word
ORDER BY word_count DESC
''').show()

+----------------+----------+
|            word|word_count|
+----------------+----------+
|        #bigdata|      7316|
|             the|      4242|
|              to|      2874|
|             #ai|      2803|
|             and|      2237|
|              of|      2135|
|               -|      2051|
|              in|      1860|
|            #iot|      1787|
|                |      1730|
|               a|      1614|
|            data|      1548|
|             for|      1436|
|#machinelearning|      1321|
|              is|      1231|
|    #datascience|      1227|
|      #analytics|      1210|
|            with|       917|
|              on|       905|
|             how|       815|
+----------------+----------+
only showing top 20 rows



7. Restituire i tweets i cui hashtag contengono "machinelearning". Ordinare i risultati in base al numero di caratteri del testo del tweet, in modo decrescente.

In [16]:
# Creiamo una UDF che, dato un array di hashtags, converte ciascuna stringa in minuscolo e
# restituisce la nuova lista di hashtags.
# Prima importiamo i tipi necessari
from pyspark.sql.types import StringType, ArrayType

def lower_hashtags(hashtags):
    return [hashtag.lower() for hashtag in hashtags]

spark.udf.register('lower_hashtags', lower_hashtags, ArrayType(StringType()))

spark.sql('''
SELECT hashtags_lower, text, length(text) AS text_length
FROM (
    SELECT *, lower_hashtags(hashtags) AS hashtags_lower
    FROM tweets
)
WHERE array_contains(hashtags_lower, 'machinelearning')
ORDER BY text_length DESC
''').show()

+--------------------+--------------------+-----------+
|      hashtags_lower|                text|text_length|
+--------------------+--------------------+-----------+
|[selfie, thinkau,...|@ozyind @Cmdr_Had...|        326|
|[bigdata, cloudna...|@CloudExpo We’ll ...|        315|
|[bigdata, analyti...|@schmarzo Bill Sc...|        313|
|[salarysurvey, da...|Are you a Data Sc...|        312|
|[blockchain, ai, ...|Blockchains for A...|        311|
|[bigdata, devops,...|@Mcl1232 SUPER EA...|        310|
|[greenx, blockcha...|#GreenX \n#Blockc...|        309|
|[virtualassistant...|In near future of...|        309|
|[bottomfish, bigd...|#BottomFish 08: $...|        308|
|[blockchain, ai, ...|Blockchains for A...|        308|
|[bigdata, cloudna...|@ExpoDX Top 200 D...|        308|
|[data, iot, bigda...|Unlock real insig...|        308|
|[cio, bigdata, an...|Bill Schmarzo Ann...|        307|
|[bigdata, analyti...|#BigData #Analyti...|        307|
|[bigdata, analyti...|#BigData #Analyti...|     